##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [ ]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [ ]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [ ]:
saut="\n"

###Préparation des matrices de traits

In [ ]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [ ]:
samplePrefix="MGC-150812-3000-1508140124-S-paradigmes"
sampleFile=samplePrefix+".csv"
goldFile="MGC-150815-total-1508150555-paradigmes.csv"
analysisPrefix="MGC-150812-3000-1508140124-S"
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

###Préparation des cases du paradigme

In [ ]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [ ]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [ ]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [ ]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [ ]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [ ]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print (minLongueur, maxLongueur, file=logfile)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print (phonemes, file=logfile)
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print ("patron1", patron, file=logfile)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print ("patron2", patron, file=logfile)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

#Classe pour la gestion des patrons, des classes et des transformations

In [ ]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [ ]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [ ]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [ ]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [ ]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]

In [ ]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [ ]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [ ]:
GOLD=pd.read_csv(goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.tolist()
goldCases.remove(u"lexeme")
for case in goldCases:
    GOLD[case]=GOLD[case].apply(lambda x: recoder(x))
    
paradigmesGOLD=GOLD[paradigmes.columns.values.tolist()]
paradigmesGOLD

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [ ]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [ ]:
paradigmes.stack().value_counts(dropna=True).sum()

In [ ]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

###Comparer les cases analysées avec l'ensemble de toutes les cases

In [ ]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

In [ ]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [ ]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [ ]:
paradigmes[paradigmes["lexeme"]=="dormir"].columns[paradigmes[paradigmes["lexeme"]=="dormir"].notnull().iloc[0]].tolist()

def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in sampleCases:
        if not pd.isnull(paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]):
            formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
            if debug: print (caseDepart,formeDepart, file=logfile)
            if formeDepart!="nan":
                for case in casesTotales:
                    if debug: print (case, file=logfile)
                    if not isinstance(resultatsLecture[(caseDepart, case)],str):
                        if "," in formeDepart:
                            formesDepart=formeDepart.split(",")
                            coef=1.0/len(formesDepart)
                            for element in formesDepart:
                                candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                        else:
                            candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
                    else: 
                        if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [ ]:
def ajouterPoint(lexeme,forme,case):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [ ]:
def generateParadigms(generation1):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortie=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortie:
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie))
    return candidats

In [ ]:
def generate(lexeme):
    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    generation2=generateParadigms(generation1)
#    print ("génération 3")
    return generation2

In [ ]:
len(paradigmes.dropna(thresh=6)["lexeme"])

In [ ]:
%%time
debug=False
listeTest=[u"manger",u"boire",u"dormir",u"aller",u"avoir"]
listeTest=paradigmes.dropna(thresh=3)["lexeme"].values.tolist()
digraphe=nx.DiGraph()
graphe=nx.Graph()
for i,element in enumerate(listeTest):
    print (i,end=", ")
    generate(element)

In [ ]:
%%time
versionStamp=dateheure()
nx.write_dot(digraphe,u"digraphe-%s.dot"%versionStamp)
nx.write_dot(graphe,u"graphe-%s.dot"%versionStamp)

In [ ]:
%%time
cliques=list(nx.algorithms.clique.find_cliques(graphe))

In [ ]:
for clique in sorted(cliques,key=lambda x: len(x),reverse=True)[:50]:
    print (len(clique),end=", ")
    print (clique)

In [ ]:
nombreElements={element:paradigmes[paradigmes["lexeme"]==element].stack().value_counts(dropna=True).sum()-1 for element in listeTest}
for element in sorted(nombreElements,key=nombreElements.get, reverse=True):
    print (element, nombreElements[element])

##Comparer la sortie des cliques avec le paradigme de départ

In [278]:
%%time
seuilClique=5
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        lexeme=clique[0].split("-")[0]
        paradigmeClique["lexeme"]=lexeme
        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
        casesLexeme.remove("lexeme")
        for element in casesLexeme:
            pointCase=u"%s-%s-%s"% (lexeme,paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0],element)
            if debug: print (pointCase, clique)
            if not pointCase in clique:
                sampleOK=False        
        if sampleOK:
            for element in clique:
                (lexeme,forme,taminfo)=element.split("-")
                paradigmeClique[taminfo]=forme
                if taminfo in paradigmes.columns:
                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
                            sampleOK=False
                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist())
print (seuilClique)

5
CPU times: user 1min 14s, sys: 182 ms, total: 1min 14s
Wall time: 1min 14s


In [279]:
paradigmesCLIQUES

,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,être,NaN,NaN,NaN,NaN,NaN,fy,s6rô,s6rE,s6rE,...,NaN,NaN,NaN,NaN,NaN,swa,NaN,swa,swa,swa
1,avoir,NaN,NaN,NaN,NaN,NaN,y,Orô,OrE,OrE,...,NaN,NaN,y,y,NaN,NaN,EjE,NaN,NaN,NaN
2,faire,NaN,NaN,NaN,fi,NaN,fi,NaN,f6rE,NaN,...,fEt,fEt,NaN,fE,NaN,fas,NaN,NaN,NaN,fas
3,dire,NaN,di,NaN,NaN,NaN,di,NaN,dirE,NaN,...,dit,NaN,NaN,di,NaN,diz,NaN,NaN,diz,diz
4,aller,NaN,NaN,NaN,NaN,NaN,ala,NaN,NaN,irE,...,NaN,alE,NaN,alE,NaN,aj,NaN,NaN,NaN,aj
5,devoir,NaN,NaN,NaN,NaN,NaN,dy,NaN,NaN,d6vrE,...,NaN,NaN,NaN,dy,NaN,NaN,NaN,NaN,dwav,dwav
6,pouvoir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,purE,purE,...,NaN,NaN,NaN,NaN,NaN,pHis,NaN,NaN,pHis,NaN
7,aimer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Em6rE,NaN,...,NaN,EmE,NaN,EmE,NaN,Em,NaN,Em,NaN,Em
8,voir,NaN,NaN,NaN,vi,NaN,vi,NaN,NaN,vErE,...,vy,vy,vy,vy,NaN,NaN,NaN,NaN,vwa,vwa
9,venir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vjêdrE,NaN,...,NaN,v6ny,v6ny,v6ny,NaN,vjEn,NaN,NaN,vjEn,NaN


%%time
seuilClique=0
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        lexeme=clique[0].split("-")[0]
        paradigmeClique["lexeme"]=lexeme
        for element in clique:
            (lexeme,forme,taminfo)=element.split("-")
            paradigmeClique[taminfo]=forme
            if taminfo in paradigmes.columns:
                if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
                    if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
                        sampleOK=False
                        print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
        if sampleOK:
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist())
print (seuilClique)

In [ ]:
#paradigmesCLIQUES
for element in listeTest:
    if paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()>1:
        print (element,paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==element].stack().value_counts(dropna=True).sum()-1)


In [ ]:
GOLD.loc[GOLD["lexeme"]=="dormir"].stack()

In [284]:
def compareGOLD(row):
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    print (row["lexeme"],end=" : ")
    lexeme=row["lexeme"]
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
        if not case in row:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
            identiques.append(u"%s : %s" % (case,row[case]))
            if debug: print ("valeurs id")
        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: print (", ".join(differents))
    elif missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(over))
    else:
        conjugaisons[lexeme]["ok"].append(identiques)
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug1: print (", ".join(identiques))
#    print ()
    return lexical

In [285]:
%%time
debug=False
debug1=False
saut="\n"
conjugaisons={}
print (", ".join(sampleCases))
paradigmesCLIQUES.apply(compareGOLD,axis=1)

ai1S, ai2S, ai3P, ai3S, fi1P, fi1S, fi2P, fi2S, fi3P, fi3S, ii1P, ii1S, ii2P, ii2S, ii3P, ii3S, inf, is1S, is3P, is3S, pI1P, pI2P, pI2S, pP, pc1P, pc1S, pc2P, pc2S, pc3P, pc3S, pi1P, pi1S, pi2P, pi2S, pi3P, pi3S, ppFP, ppFS, ppMP, ppMS, ps1P, ps1S, ps2P, ps2S, ps3P, ps3S
être : avoir : faire : dire : aller : devoir : pouvoir : aimer : voir : venir : trouver : prendre : approuver : approuver : partir : prouver : prouver : penser : retrouver : savoir : retrouver : éprouver : éprouver : vouloir : dépenser : approuver : supposer : supposer : estimer : taper : cesser : remplacer : bouffer : bosser : parler : déposer : donner : prouver : redresser : redresser : débarrasser : débarrasser : attendre : retrouver : danser : exposer : exposer : fumer : imposer : disposer : disposer : recommencer : embrasser : attraper : éprouver : laisser : adresser : adresser : adresser : rincer : écraser : enfoncer : proposer : commander : céder : douter : douter : agacer : agacer : annoncer : approuver : organ

In [297]:
pb=[i for i in conjugaisons if conjugaisons[i]["diff"]]
for element in pb:
    print (element)
    print ("DIFF",conjugaisons[element]["diff"])
    print ("MISS",conjugaisons[element]["miss"],end="\n\n")

projeter
DIFF [[u'ps1S : prOZ6t \u2260 prOZEt', u'pI2S : prOZ6t \u2260 prOZEt', u'pi1S : prOZ6t \u2260 prOZEt', u'pi3S : prOZ6t \u2260 prOZEt'], [u'pi1S : prOZ6t \u2260 prOZEt']]
MISS [[u'ii1P : \xd8 \u2260 prOZ6tj\xf4', u'pP : \xd8 \u2260 prOZ6t\xe2', u'pc1P : \xd8 \u2260 prOZEt6rj\xf4', u'ii1S : \xd8 \u2260 prOZ6tE', u'is3P : \xd8 \u2260 prOZ6tas', u'is3S : \xd8 \u2260 prOZ6ta', u'ps3S : \xd8 \u2260 prOZEt', u'ii3P : \xd8 \u2260 prOZ6tE', u'pi2S : \xd8 \u2260 prOZEt', u'ai1S : \xd8 \u2260 prOZ6tE', u'pi2P : \xd8 \u2260 prOZ6tE', u'ppFP : \xd8 \u2260 prOZ6tE', u'ps1P : \xd8 \u2260 prOZ6tj\xf4', u'ps3P : \xd8 \u2260 prOZEt', u'ps1S : \xd8 \u2260 prOZEt', u'pI2S : \xd8 \u2260 prOZEt', u'fi2P : \xd8 \u2260 prOZEt6rE', u'fi2S : \xd8 \u2260 prOZEt6ra', u'ps2P : \xd8 \u2260 prOZ6tjE', u'ps2S : \xd8 \u2260 prOZEt', u'ai2S : \xd8 \u2260 prOZ6ta', u'pc2P : \xd8 \u2260 prOZEt6rjE', u'pc2S : \xd8 \u2260 prOZEt6rE', u'ii2P : \xd8 \u2260 prOZ6tjE', u'pi1S : \xd8 \u2260 prOZEt', u'pi1P : \xd8 \u226

In [ ]:
paradigmes[paradigmes["lexeme"]==u"éprouver"].stack()

In [ ]:
GOLD[GOLD["lexeme"]==u"éprouver"]

In [ ]:
paradigmesCLIQUES[paradigmesCLIQUES["lexeme"]==u"éprouver"].stack()

In [ ]:
paradigmesGOLD

In [ ]:
paradigmesGOLD.loc[paradigmesGOLD["lexeme"]=="boire"]["inf"]

##Faire un tableau XML pour la sortie des cliques

class tableauParadigme:
    '''
    Gestion des tableaux de flexion
    '''

    def __init__(self):
        self.paradigme={}
        self.lexeme=""

    def ajouterElement(self,element):
        (lexeme,forme,taminfo)=element.split("-")
        if self.lexeme=="":
            self.lexeme=lexeme
        if self.lexeme==lexeme:
            if len(taminfo)==4:
                (tense,mode,pers,num)=taminfo
            elif taminfo=="inf":
                (tense,mode,pers,num)=("",taminfo,"","")
                
            elif taminfo=="pP":
                (tense,mode,pers,num)=("p","part","","")
            else:
                print("erreur case",element,file=logfile)
            if not mode in self.paradigme:
                self.paradigme[mode]={}
            if not tense in self.paradigme[mode]:
                self.paradigme[mode][tense]={}
            if not pers+num in self.paradigme[mode][tense]:
                self.paradigme[mode][tense][pers+num]=[]
            self.paradigme[mode][tense][pers+num].append(forme)
            
      

seuilClique=25

for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
    if len(clique)>seuilClique:
        print (n, len(clique))
        tabParadigme=tableauParadigme()
        for element in clique:
            tabParadigme.ajouterElement(element)
        for mode in ["i","s","c","I"]:
            for tense in tabParadigme.paradigme[mode]:
                if mode=="I":
                    for pers in ["2S", "1P", "2P"]:
                        print ("-".join(tabParadigme.paradigme[mode][tense][pers]), end=", ")
                else:
                    for pers in [str(i+1)+"S" for i in range(3)]+[str(i+1)+"P" for i in range(3)]:
                        print ("-".join(tabParadigme.paradigme[mode][tense][pers]), end=", ")
                print()